In [5]:
from bs4 import BeautifulSoup
import os
import csv

def remove_html_tags(text):
    soup = BeautifulSoup(text, 'html.parser')
    return soup.get_text()

def read_html_file(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        lines = file.readlines()
    return lines

folder_path = 'raw_html'  # Specify the folder path containing the HTML files

output_folder = 'clean_text'  # Specify the folder where the clean text files will be saved
os.makedirs(output_folder, exist_ok=True)  # Create the output folder if it doesn't exist

html_files = os.listdir(folder_path)  # Get the list of HTML files in the folder

for html_file in html_files:
    html_file_path = os.path.join(folder_path, html_file)
    lines = read_html_file(html_file_path)
    html_text = ''.join(lines)  # Join the lines into a single string
    clean_text = remove_html_tags(html_text)

    # Extract the filename from the html_file_path
    filename = os.path.splitext(html_file)[0]  # Extract the filename without extension
    output_file_path = os.path.join(output_folder, f'clean_text_{filename}.csv')

    with open(output_file_path, 'w', newline='', encoding='utf-8') as output_file:
        writer = csv.writer(output_file)
        for line in clean_text.splitlines():
            writer.writerow([line])

    print(f"Clean text saved in {output_file_path}.")


Clean text saved in clean_text\clean_text_Criminal Justice and Police Act 2001 (c. 16).csv.
Clean text saved in clean_text\clean_text_Human Rights Act 1998 (c. 42).csv.
Clean text saved in clean_text\clean_text_Public Order Act 1986 (c. 64).csv.
Clean text saved in clean_text\clean_text_Public Order Act 2023 (c. 15).csv.
Clean text saved in clean_text\clean_text_Serious Organised Crime and Police Act 2005 (c. 15).csv.


In [21]:
import csv
import pandas as pd
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from gensim import corpora, models

# Load stopwords
stop_words = set(stopwords.words('english'))

# Function to preprocess text
def preprocess_text(text):
    tokens = word_tokenize(text.lower())
    tokens = [token for token in tokens if token.isalpha() and token not in stop_words]
    return tokens

# Function to perform topic modeling on a list of tokens
def perform_topic_modeling(tokens):
    if not tokens:
        return []  # Return empty topics if there are no tokens
    dictionary = corpora.Dictionary([tokens])
    corpus = [dictionary.doc2bow(tokens)]
    lda_model = models.LdaModel(corpus, num_topics=1, id2word=dictionary)
    topics = lda_model.print_topics(num_words=5)
    return [topic[1] for topic in topics]

# Read CSV file
data = []
with open('clean_text\\clean_text_Human Rights Act 1998 (c. 42).csv', 'r', encoding='utf-8') as file:
    reader = csv.reader(file)
    for row in reader:
        text = row[0]  # Assuming the text is in the first column
        tokens = preprocess_text(text)
        topics = perform_topic_modeling(tokens)
        data.append([text, topics])

# Create DataFrame
df = pd.DataFrame(data, columns=['Text', 'Topics'])

# Save DataFrame to CSV file
df.to_csv('output.csv', index=False)


Below is the search query without using word vectorization

In [33]:
import pandas as pd

# Configure pandas display options
pd.set_option('display.max_colwidth', None)

# Load the output.csv file into a DataFrame
df = pd.read_csv('output.csv')

# Function to search the DataFrame based on the query
def search_dataframe(query):
    # Filter rows that match the query (case-insensitive)
    matching_rows = df[df['Topics'].str.contains(query, case=False)]
    return matching_rows

# Get the query from user input
query = input("Enter your query: ")

# Search the DataFrame
results = search_dataframe(query)

# Print the query text in a pretty way
print(f"Query: {query}\n")

# Display the matching results without the Topics column
print("Matching Results:")
print(results.drop(columns=['Topics']).to_string(index=False))


Query: damage

Matching Results:
                                                                                                                                          Text
(2)But damages may be awarded only by a court which has power to award damages, or to order the payment of compensation, in civil proceedings.
                                  (3)No award of damages is to be made unless, taking account of all the circumstances of the case, including—
                                                                                                               (a)whether to award damages, or
                                                                     (5)A public authority against which damages are awarded is to be treated—
                                                                        “damages” means damages for an unlawful act of a public authority; and
              [F16(3)In proceedings under this Act in respect of a judicial act done in good faith, damages m

Below is the search using word vectarization with the google training dataset

In [16]:
import pandas as pd
from gensim.models import KeyedVectors
from gensim.utils import simple_preprocess

# Configure pandas display options
pd.set_option('display.max_colwidth', None)

# Load the output.csv file into a DataFrame
df = pd.read_csv('output.csv')

# Load the Word2Vec model (pre-trained or trained on your data)
model_path = 'GoogleNews-vectors-negative300.bin'
word2vec_model = KeyedVectors.load_word2vec_format(model_path, binary=True)

In [18]:
# Function to expand the query with similar words
def expand_query(query):
    expanded_query = []
    for word in simple_preprocess(query):
        similar_words = word2vec_model.most_similar(word)
        expanded_query.extend([word] + [similar_word[0] for similar_word in similar_words])
    return ' '.join(expanded_query)

# Function to search the DataFrame based on the expanded query
def search_dataframe(query):
    # Split the query into individual words
    words = query.split()
    
    # Initialize an empty list to store matching DataFrames
    matching_dfs = []
    
    # Search for each word in the query
    for word in words:
        # Filter rows that match the word (case-insensitive)
        matching_rows = df[df['Topics'].str.contains(word, case=False, regex=False)]
        # Append the matching DataFrame to the list
        matching_dfs.append(matching_rows)
    
    # Concatenate all the matching DataFrames into a single DataFrame
    results = pd.concat(matching_dfs, ignore_index=True)
    
    return results

# Get the query from user input
query = input("Enter your query: ")

# Expand the query with similar words
expanded_query = expand_query(query)

# Search the DataFrame
results = search_dataframe(expanded_query)

# Print the original and expanded queries in a pretty format
print(f"Original Query: {query.capitalize()}")
print(f"Expanded Query: {expanded_query.capitalize()}\n")

# Display the matching results without the Topics column
print("Matching Results:")
print(results.drop(columns=['Topics']).to_string(index=False))


Original Query: Attack
Expanded Query: Attack attacks attck atack assault atttack bombing assaults attacking attacked counterattack

Matching Results:
                                                                                                                                                                                                                                                                                                                         Text
(7)“Overseas operations” means any operations outside the British Islands, including peacekeeping operations and operations for dealing with terrorism, civil unrest or serious public disorder, in the course of which members of Her Majesty's forces come under attack or face the threat of attack or violent resistance.
